In [2]:
!pip install faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [1]:
# Import necessary libraries

import json
import joblib
from collections import defaultdict
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import scipy.sparse

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import requests
url = 'https://northeastern-my.sharepoint.com/personal/mhatre_ag_northeastern_edu/_layouts/15/download.aspx?UniqueId=3e78d9923545475397681c5e8a47383d&e=gTeFhD'
r = requests.get(url, allow_redirects=True)
url1 = 'https://northeastern-my.sharepoint.com/personal/mhatre_ag_northeastern_edu/_layouts/15/download.aspx?UniqueId=a6e25a75d96e42139fc5b23ba7a7bc1b&e=S9FI43'
r = requests.get(url1, allow_redirects=True)

## Step 1: Join business dataset with reviews dataset. Tokenize reviews and convert to lowercase

In [4]:
# Store reviews for each business
import random
business_information = defaultdict(lambda: {"reviews": []})

with open("/content/drive/MyDrive/yelp_academic_dataset_review.json", "r", encoding="utf-8") as file:
    for line in file:
        # Keep only 10% of the lines
        if random.random() <= 0.10:
            curr = json.loads(line)
            business_information[curr["business_id"]]["reviews"].append(curr["text"])

In [5]:
# Extract business-specific data and store in dictionary
with open("/content/drive/MyDrive/yelp_academic_dataset_business.json", "r", encoding="utf-8") as file:

    for line in file:

        curr = json.loads(line)
        business_information[curr["business_id"]]["name"] = curr["name"]
        business_information[curr["business_id"]]["address"] = curr["address"]
        business_information[curr["business_id"]]["city"] = curr["city"]
        business_information[curr["business_id"]]["state"] = curr["state"]
        business_information[curr["business_id"]]["latitude"] = curr["latitude"]
        business_information[curr["business_id"]]["longitude"] = curr["longitude"]
        business_information[curr["business_id"]]["stars"] = curr["stars"]
        business_information[curr["business_id"]]["categories"] = curr["categories"]

In [6]:
# Tokenize reviews and convert all tokens to lowercase
tokenized_business_information = defaultdict(lambda: {"reviews": []})

for id_ in business_information.keys():

    for review in business_information[id_]["reviews"]:

      if random.random() <= 0.10:

        tokenized_business_information[id_]["reviews"].append([word.lower() for word in word_tokenize(review)])

In [7]:
# Write joined, tokenized business/review data to a file
with open('business_with_reviews.json', 'w') as f:
    for key, value in tokenized_business_information.items():
        json.dump({key: value}, f)
        f.write('\n')

## B+

2 Problems:
1. Can't extract user lat or lon
2. Have to hardcode every bordering state

In [40]:
user_state = "CA"
user_lat = 37.77   # San Francisco
user_lon = -122.42
# Hardcode every brodering state
bordering_states = {
    'AL': ['FL', 'GA', 'MS', 'TN'],
    'AK': [],
    'AZ': ['CA', 'CO', 'NM', 'NV', 'UT'],
    'AR': ['LA', 'MO', 'MS', 'OK', 'TN', 'TX'],
    'CA': ['AZ', 'NV', 'OR'],
    'CO': ['AZ', 'KS', 'NE', 'NM', 'OK', 'UT', 'WY'],
    'CT': ['MA', 'NY', 'RI'],
    'DE': ['MD', 'NJ', 'PA'],
    'FL': ['AL', 'GA'],
    'GA': ['AL', 'FL', 'NC', 'SC', 'TN'],
    'HI': [],
    'ID': ['MT', 'NV', 'OR', 'UT', 'WA', 'WY'],
    'IL': ['IA', 'IN', 'KY', 'MO', 'WI'],
    'IN': ['IL', 'KY', 'MI', 'OH'],
    'IA': ['IL', 'MN', 'MO', 'NE', 'SD', 'WI'],
    'KS': ['CO', 'MO', 'NE', 'OK'],
    'KY': ['IL', 'IN', 'MO', 'OH', 'TN', 'VA', 'WV'],
    'LA': ['AR', 'MS', 'TX'],
    'ME': ['NH'],
    'MD': ['DE', 'PA', 'VA', 'WV'],
    'MA': ['CT', 'NH', 'NY', 'RI', 'VT'],
    'MI': ['IN', 'OH', 'WI'],
    'MN': ['IA', 'ND', 'SD', 'WI'],
    'MS': ['AL', 'AR', 'LA', 'TN'],
    'MO': ['AR', 'IA', 'IL', 'KS', 'KY', 'NE', 'OK', 'TN'],
    'MT': ['ID', 'ND', 'SD', 'WY'],
    'NE': ['CO', 'IA', 'KS', 'MO', 'SD', 'WY'],
    'NV': ['AZ', 'CA', 'ID', 'OR', 'UT'],
    'NH': ['MA', 'ME', 'VT'],
    'NJ': ['DE', 'NY', 'PA'],
    'NM': ['AZ', 'CO', 'OK', 'TX', 'UT'],
    'NY': ['CT', 'MA', 'NJ', 'PA', 'VT'],
    'NC': ['GA', 'SC', 'TN', 'VA'],
    'ND': ['MN', 'MT', 'SD'],
    'OH': ['IN', 'KY', 'MI', 'PA', 'WV'],
    'OK': ['AR', 'CO', 'KS', 'MO', 'NM', 'TX'],
    'OR': ['CA', 'ID', 'NV', 'WA'],
    'PA': ['DE', 'MD', 'NJ', 'NY', 'OH', 'WV'],
    'RI': ['CT', 'MA'],
    'SC': ['GA', 'NC'],
    'SD': ['IA', 'MN', 'MT', 'ND', 'NE', 'WY'],
    'TN': ['AL', 'AR', 'GA', 'KY', 'MO', 'MS', 'NC', 'VA'],
    'TX': ['AR', 'LA', 'NM', 'OK'],
    'UT': ['AZ', 'CO', 'ID', 'NV', 'NM', 'WY'],
    'VT': ['MA', 'NH', 'NY'],
    'VA': ['KY', 'MD', 'NC', 'TN', 'WV'],
    'WA': ['ID', 'OR'],
    'WV': ['KY', 'MD', 'OH', 'PA', 'VA'],
    'WI': ['IA', 'IL', 'MI', 'MN'],
    'WY': ['CO', 'ID', 'MT', 'NE', 'SD', 'UT']
}

In [41]:
from math import radians, cos, sin, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    angle = sin((lat2 - lat1)/2)**2 + cos(lat1) * cos(lat2) * sin((lon2 - lon1)/2)**2
    dist = 2 * asin(sqrt(angle))
    return dist * 6371

In [44]:
def extract_2nd(a):
    return a[1]

def rank_by_location(query, state_u, lat_u, lon_u):
    restaurant_values = []
    allowed_states = [state_u] + bordering_states.get(state_u, [])

    for bid, info in business_information.items():
        # Check if any info is empty
        lat, lon = info.get("latitude"), info.get("longitude")
        if lon is None or lat is None:
            continue
        if "reviews" not in info or not info["reviews"]:
            continue
        # Check if the state is close, if not skip
        if info.get("state") not in allowed_states:
            continue

        distance = haversine(lat_u, lon_u, lat, lon)
        value = info["stars"]

        # Find distance
        if distance > 5:
            # Only penalize the portion over 5km
            value = value - 0.005 * (distance - 5)
        restaurant_values.append((bid, value))

    return sorted(restaurant_values, key=extract_2nd, reverse=True)[:10]

In [45]:
results = rank_by_location("vegan", user_state, user_lat, user_lon)

for n, value in results:
    name = business_information[n]["name"]
    print(f"{name} ({n}): score = {value:.2f}")

Ferris Heating (z-he5WZr5Ekq-rA6AHgtqQ): score = 3.68
Discount Window Coverings (tzvbWBUJJ2rPBJTo_OyTVw): score = 3.68
Tributary Whitewater Tours (VMP_tCZXcuJuiNUCBshd8w): score = 3.67
Eastside Deli (D5n8XvQqC_vzOCi63wNChQ): score = 3.67
Raft California (_dLINFtHKdhescCZiPkPlA): score = 3.67
Saturno Home Inspection (F4cezLy3xwbzCXG4lww74w): score = 3.66
Tahoe Meadows (EUvK_xnJb_ZbMZ1T5uTBUQ): score = 3.64
Sterling Detail (s9qQ2WAvlsckxabl4AEO4w): score = 3.62
Elsa Boscarello Photography (luwAuq3yQtCFeFMxc3K-Ew): score = 3.62
Lemaire Photography (E2L5Iu25tzkQ9ig-ZAqQ5Q): score = 3.62


## Step 2: Output scores for TFIDF ranker

In [12]:
# Read in business reviews from JSON file
business_reviews = dict()

with open('business_with_reviews.json', 'r') as f:

    for line in f:

        record = json.loads(line)
        business_reviews.update(record)

In [38]:
# Test one query on the entire dataset, check results
business_ids = list(business_reviews.keys())
corpus = []
for id_ in business_ids:

    # Each business review is condensed down into a single, long sentence
    # so that TFIDF and other embedding techniques could be applied
    corpus.append(" ".join(" ".join(tokens) for tokens in business_reviews[id_]["reviews"]))

In [14]:
# Create TFIDF vector and train vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)

# Persist vector and model
joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')
scipy.sparse.save_npz("tfidf_matrix.npz", tfidf_matrix)

In [15]:
# Reinitialize vectorizer and tfidf_matrix if session is lost
vectorizer = joblib.load('tfidf_vectorizer.joblib')
tfidf_matrix = scipy.sparse.load_npz("tfidf_matrix.npz")

In [16]:
# Transform query so that it fits into TFIDF vector space
query = "cafe with great lattes in san francisco"
query_vec = vectorizer.transform([query])

# Use cosine similarity to find most similar document for the query
similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()

In [17]:
# Match up business ids with similarity scores
scores = []

n = len(business_ids)
for i in range(n): scores.append((business_ids[i], similarities[i]))

# Sort based on score; sort in descending order
scores.sort(key = lambda x: x[1], reverse=True)

In [18]:
# Print out top 10 results
for id, _ in scores[:10]:

    print(business_information[id]["name"])

Jiffy Lube
Perks Coffee & Cafe
Edible Arrangements
Philter
Envie Espresso Bar & Cafe
De Ville Coffee House & Creperie
Mitchell’s Coffee Time
Delco's Original Steaks & Hoagies
Crepe & Shake Cafe
Discount Tire


## Step 3: Create/Store Embeddings and Use FAISS to generate scores, Compare to token-matching ranker

In [21]:
!pip install torch

In [24]:
import torch

In [29]:
# Use transformer architecture to create dense vector embeddings
# from query and restaurant reviews
# all = general purpose transformer
# MiniLM = lightweight transformer similar to BERT
# L6 = six layers
# v2 = version two
model = SentenceTransformer("all-MiniLM-L6-v2")
subset_corpus = random.sample(corpus, int(0.01 * len(corpus)))
# Create embeddings from the reviews
embeddings = model.encode(
    subset_corpus,                      # List of documents
    batch_size=16,               # Tune this value (16–64 for Colab/CPU)
    convert_to_numpy=True,      # Needed for FAISS
    show_progress_bar=True,     # See how long it takes
    device='cuda' if torch.cuda.is_available() else 'cpu'  # Use GPU if available
)

dimension = embeddings.shape[1]

# Create FAISS index for similarity search
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

Batches:   0%|          | 0/24 [00:00<?, ?it/s]

In [30]:
# Persist index to file
faiss.write_index(index, "business_index.faiss")

In [31]:
# Read index back from file if session is lost
index = faiss.read_index("business_index.faiss")

In [32]:
# Check results of test query
query = "cafe with great lattes in san francisco"
query_vec = model.encode([query], convert_to_numpy=True)

# Get top k results
k = 50
D, I = index.search(query_vec, k)
top_businesses = [business_ids[i] for i in I[0]]

for id in top_businesses:

    print(business_information[id]["name"])

Mellow Mushroom - Brandon
Chipotle Mexican Grill
Surrey's Café & Juice Bar
HipCityVeg - University City
Penca
Pizzeria Vetri
Ninki
Andrea's Restaurant
DoubleTree by Hilton Hotel Philadelphia Center City
Dairy Queen
Hampton Inn St. Louis/Sunset Hills
Hotel Mazarin
Honey's Sit-N-Eat
Dooners
The Groomin Room
Kimpton Hotel Monaco Philadelphia
The Cake Bake Shop by Gwendolyn Rogers-Broad Ripple Village
1 Tippling Place
Copabanana
Crazy Sushi
China Kitchen
Indique Virgin Hair Extensions
Citizens Bank Park
Cafe Beignet on Bourbon Street
Bellagio Pizza & Subs
Kettle Restaurant
Roscoe's Tacos - Greenwood
Tony's Luke
Sushi Tyme
National Underground
Mahony's Po-Boys & Seafood
Mayflower
Layla's Honky Tonk
Mena's Palace
Bernie's Restaurant & Bar - Hatboro
Gateway To India
B2 Bluefin
Cal Taco
Mother Nature's Cupboard
Tryst Gastro Lounge
The Home Depot
Boostin' Bowls
Wheel Tec
3500 Winehaus
Sam's Beach Bar
Children's Hospital of Philadelphia
Tampa Bay Brewing Company Westchase
Ants Pants Café
RumFish

B+ with embeddings

In [54]:
from math import radians, cos, sin, asin, sqrt

# Haversine to compute distance in kilometers
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    angle = sin((lat2 - lat1)/2)**2 + cos(lat1) * cos(lat2) * sin((lon2 - lon1)/2)**2
    dist = 2 * asin(sqrt(angle))
    return dist * 6371

# Extract sorting key
def extract_2nd(a):
    return a[1]

# Your embedding-enhanced + location-aware ranker
def rank_by_location_with_embeddings(query, state_u, lat_u, lon_u, model, index, business_ids, business_info):
    allowed_states = [state_u] + bordering_states.get(state_u, [])

    # Encode query with SentenceTransformer
    query_vec = model.encode([query], convert_to_numpy=True)
    _, I = index.search(query_vec, 100)
    top_ids = [business_ids[i] for i in I[0]]

    restaurant_values_embed = []
    for bid in top_ids:
        info = business_info.get(bid, {})
        # Check if any info is empty
        lat, lon = info.get("latitude"), info.get("longitude")
        if not lat or not lon:
            continue
        if not info.get("reviews"):
            continue
        # Check if the state is close, if not skip
        if info.get("state") not in allowed_states:
            continue

        # Find distance
        distance = haversine(lat_u, lon_u, lat, lon)
        value = info["stars"]

        if distance > 5:
            # Only penalize the portion over 5km
            value = value - 0.005 * (distance - 5)

        restaurant_values_embed.append((bid, value))

    return sorted(results, key=extract_2nd, reverse=True)[:10]

In [55]:
# Example user location
user_state = "CA"
user_lat = 37.77
user_lon = -122.42

# Query and rank
results = rank_by_location_with_embeddings(
    "vegan burger", user_state, user_lat, user_lon,
    model, index, business_ids, business_information
)

# Print top results
for bid, score in results:
    print(f"{business_information[bid]['name']} ({bid}): score = {score:.2f}")

Cold Spring Tavern (0qu0fNTOsSmuREYVIMPuIQ): score = 2.38
Santa Barbara County Courthouse (6JFTijOMHB46yBoyVOjPCA): score = 2.30
Rickshaw Sushi (yqRF449dYddbt1bDg50J_Q): score = 2.04
Western Village Inn and Casino (_E0WPHfJEYmwOUCAj-Jq1A): score = 2.00
Cal Taco (VeFfrEZ4iWaecrQg6Eq4cg): score = 1.85
West Beach Inn, a Coast Hotel (7j8rT3FZbXgL4k0DnXi2Mw): score = 1.79
The Home Depot (41QlNQkkCVvmzOPSFFz1FA): score = 1.05
Panda Express (UKA2eeAZmO4ACB2J0kFISw): score = 1.04
Prep & Pastry (UCMSWPqzXjd7QHq7v8PJjQ): score = -1.53
Sher-e-Punjab (f82dhKNiUXsDVPMLqKYiIQ): score = -2.02
